In [ ]:
# Code to mount google drive in case you are loading the data from your google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
import os 
data_path = '/gdrive/MyDrive/AppliedAI_ThesisProj/dataset'
os.chdir(data_path)
print(os.getcwd())

In [ ]:
!pip3 install scikit-image

In [ ]:
import os, sys, math, io
import numpy as np
import pandas as pd
import multiprocessing as mp
import bson
import struct

%matplotlib inline
import matplotlib.pyplot as plt
import skimage.io
import keras
from keras.preprocessing.image import load_img, img_to_array
# Check on Keras Preprocessing
#from keras_preprocessing import image
import tensorflow as tf

from collections import defaultdict
from tqdm import *

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../cdiscount-image-classification-challenge"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

In [ ]:
data_dir = "../cdiscount-image-classification-challenge"

train_bson_path = os.path.join(data_dir, "train.bson")
num_train_products = 7069896

# train_bson_path = os.path.join(data_dir, "train_example.bson")
# num_train_products = 82

test_bson_path = os.path.join(data_dir, "test.bson")
num_test_products = 1768182

# Create lookup tables

In [ ]:
#The generator uses several lookup tables that describe the layout of the BSON file, which products and images are part of the training/validation sets, and so on.

#You only need to generate these tables once, as they get saved to CSV files.

#Lookup table for categories
categories_path = os.path.join(data_dir, "category_names.csv")
categories_df = pd.read_csv(categories_path, index_col="category_id")

# Maps the category_id to an integer index. This is what we'll use to
# one-hot encode the labels.
categories_df["category_idx"] = pd.Series(range(len(categories_df)), index=categories_df.index)

categories_df.to_csv("categories.csv")
categories_df.head()

In [ ]:
#Create dictionaries for quick lookup of category_id to category_idx mapping.
def make_category_tables():
    cat2idx = {}
    idx2cat = {}
    i=0
    for ir in categories_df.itertuples():
            
        category_id = ir[0]
        category_idx = ir[4]
        cat2idx[category_id] = category_idx
        idx2cat[category_idx] = category_id
      
    return cat2idx, idx2cat

In [ ]:

cat2idx, idx2cat = make_category_tables()

In [ ]:
# Test if it works:
cat2idx[1000012755], idx2cat[4]

In [ ]:
#this takes a few minutes to execute, but we only have to do it once (we'll save the table to a CSV file afterwards).
def read_bson(bson_path, num_records, with_categories):
    rows = {}
    with open(bson_path, "rb") as f, tqdm(total=num_records) as pbar:
        offset = 0
        while True:
            item_length_bytes = f.read(4)
            if len(item_length_bytes) == 0:
                break

            length = struct.unpack("<i", item_length_bytes)[0]

            f.seek(offset)
            item_data = f.read(length)
            assert len(item_data) == length

            item = bson.BSON.decode(item_data)
            product_id = item["_id"]
            num_imgs = len(item["imgs"])

            row = [num_imgs, offset, length]
            if with_categories:
                row += [item["category_id"]]
            rows[product_id] = row

            offset += length
            f.seek(offset)
            pbar.update()

    columns = ["num_imgs", "offset", "length"]
    if with_categories:
        columns += ["category_id"]

    df = pd.DataFrame.from_dict(rows, orient="index")
    df.index.name = "product_id"
    df.columns = columns
    df.sort_index(inplace=True)
    return df

In [ ]:
%time train_offsets_df = read_bson(train_bson_path, num_records=num_train_products, with_categories=True)

In [ ]:
train_offsets_df.columns

In [ ]:
train_offsets_df.head()

In [ ]:
train_offsets_df['category_id'].value_counts()

In [ ]:
train_offsets_df['num_imgs'].value_counts()

In [ ]:
train_offsets_df.to_csv("train_offsets.csv")

## Create a random train/validation split
We split on products, not on individual images. Since some of the categories only have a few products, we do the split separately for each category.

This creates two new tables, one for the training images and one for the validation images. There is a row for every single image, so if a product has more than one image it occurs more than once in the table.

In [ ]:
def make_val_set(df, split_percentage=0.2, drop_percentage=0.):
    # Find the product_ids for each category.
    category_dict = defaultdict(list)
    for ir in tqdm(df.itertuples()):
        category_dict[ir[4]].append(ir[0])

    train_list = []
    val_list = []
    with tqdm(total=len(df)) as pbar:
        for category_id, product_ids in category_dict.items():
            category_idx = cat2idx[category_id]

            # Randomly remove products to make the dataset smaller.
            keep_size = int(len(product_ids) * (1. - drop_percentage))
            if keep_size < len(product_ids):
                product_ids = np.random.choice(product_ids, keep_size, replace=False)

            # Randomly choose the products that become part of the validation set.
            val_size = int(len(product_ids) * split_percentage)
            if val_size > 0:
                val_ids = np.random.choice(product_ids, val_size, replace=False)
            else:
                val_ids = []

            # Create a new row for each image.
            for product_id in product_ids:
                row = [product_id, category_idx]
                for img_idx in range(df.loc[product_id, "num_imgs"]):
                    if product_id in val_ids:
                        val_list.append(row + [img_idx])
                    else:
                        train_list.append(row + [img_idx])
                pbar.update()
                
    columns = ["product_id", "category_idx", "img_idx"]
    train_df = pd.DataFrame(train_list, columns=columns)
    val_df = pd.DataFrame(val_list, columns=columns)   
    return train_df, val_df

In [ ]:
train_images_df, val_images_df = make_val_set(train_offsets_df, split_percentage=0.2, 
                                              drop_percentage=0.6)

In [ ]:
train_images_df.head()

In [ ]:
val_images_df.head()

In [ ]:
print("Number of training images:", len(train_images_df))
print("Number of validation images:", len(val_images_df))
print("Total images:", len(train_images_df) + len(val_images_df))

In [ ]:
len(train_images_df["category_idx"].unique()), len(val_images_df["category_idx"].unique())

In [ ]:
train_images_df.to_csv("train_images.csv")
val_images_df.to_csv("val_images.csv")

## Part 2: The generator

In [ ]:
#First load the lookup tables from the CSV files (you don't need to do this if you just did all the steps from part 1).

categories_df = pd.read_csv("categories.csv", index_col=0)
# cat2idx, idx2cat = make_category_tables()

train_offsets_df = pd.read_csv("train_offsets.csv", index_col=0)
train_images_df = pd.read_csv("train_images.csv", index_col=0)
val_images_df = pd.read_csv("val_images.csv", index_col=0)


In [ ]:
from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from PIL import Image
from pymongo import MongoClient
import io
import pymongo
import skimage.io as skio
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array

client = MongoClient(connect= False)
train = client.Cdiscount['train']
test = client.Cdiscount['test']


class BSONIterator(Iterator):
    def __init__(self, bson_file, images_df, offsets_df, num_class,
                 image_data_generator, lock, target_size=(180, 180), 
                 with_labels=True, batch_size=32, shuffle=False, seed=None):

        self.file = bson_file
        self.images_df = images_df
        self.offsets_df = offsets_df
        self.with_labels = with_labels
        self.samples = len(images_df)
        self.num_class = num_class
        self.image_data_generator = image_data_generator
        self.target_size = tuple(target_size)
        self.image_shape = self.target_size + (3,)

        print("Found %d images belonging to %d classes." % (self.samples, self.num_class))

        super(BSONIterator, self).__init__(self.samples, batch_size, shuffle, seed)
        self.lock = lock

    def _get_batches_of_transformed_samples(self, index_array):
        batch_x = np.zeros((len(index_array),) + self.image_shape, dtype=K.floatx())
        if self.with_labels:
            batch_y = np.zeros((len(batch_x), self.num_class), dtype=K.floatx())

        for i, j in enumerate(index_array):
            # Protect file and dataframe access with a lock.
            with self.lock:
                image_row = self.images_df.iloc[j]
                product_id = image_row["product_id"] 
                offset_row = self.offsets_df.loc[product_id]

                # Read this product's data from the BSON file.
                self.file.seek(offset_row["offset"])
                item_data = self.file.read(offset_row["length"])

                
            # Grab the image from the product.
            #print(j)
            #img_idx =0
            item = bson.BSON.decode(item_data)
            #item = train.find_one({'_id':int(j)})
            #print(item)
            img_idx = image_row["img_idx"]
            #print(img_idx)
            bson_img = item["imgs"][img_idx]["picture"]

            img = Image.open(io.BytesIO(bson_img))
            img = img.convert('RGB')
            img = img.resize(self.target_size, Image.NEAREST)
            # Preprocess the image.
            x = img_to_array(img)
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)

            # Add the image and the label to the batch (one-hot encoded).
            batch_x[i] = x
            if self.with_labels:
                batch_y[i, image_row["category_idx"]] = 1

        if self.with_labels:
            return batch_x, batch_y
        else:
            return batch_x

    def next(self):
        with self.lock:
            index_array = next(self.index_generator)
        #print(index_array)
        return self._get_batches_of_transformed_samples(index_array)

In [ ]:
train_bson_file = open(train_bson_path, "rb")

#Create a generator for training and a generator for validation.

In [ ]:
#Because the training and validation generators read from the same BSON file, they need to use the same lock to protect it.
import threading
lock = threading.Lock()

In [ ]:
num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 128
target_size = (180,180)
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df, 
                         num_classes, train_datagen, lock,target_size = target_size,
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock,target_size = target_size,
                       batch_size=batch_size, shuffle=True)

In [ ]:
#next(train_gen)  # warm-up

%time bx, by = next(train_gen)

In [ ]:
%time bx, by = next(train_gen)

In [ ]:
len(by)

In [ ]:
print(bx[0].shape)

In [ ]:
plt.imshow(bx[4].astype(np.uint8))

print("ClassID_OneHotencoded:{}".format(by))

# Modelling

In [ ]:
# Wandb Loggers
import wandb
from wandb.keras import WandbCallback
wandb.init(project="cdiscount-challenge-diploma-project", entity="udaygirish")

In [ ]:
# Import TF Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


## Xception 

In [ ]:
def make_model_exception(input_shape, num_classes):
    inputs = keras.Input(shape = input_shape)
    x = inputs
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)

    activation = "softmax"
    units = num_classes
    
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(units, activation= activation)(x)
    return keras.Model(inputs,outputs)

In [ ]:
model = make_model_exception(input_shape = (180,180,3), num_classes = 5270)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 10
base_learning_rate = 1e-2
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=base_learning_rate,
    decay_steps=20000,
    decay_rate=0.9)

wandb.config = {
 "learning_rate" : base_learning_rate,
 "epochs" : epochs,
 "batch_size" : 128
}
filepath = "./weights.best_exception_{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_f = keras.callbacks.ModelCheckpoint(filepath,monitor = "val_accuracy", verbose=1, save_best_only=False, mode="max")

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"), checkpoint_f, WandbCallback()
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate = base_learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy", precision_m, recall_m, f1_m],
)
H = model.fit(
    train_gen, epochs=epochs, callbacks=callbacks, validation_data=val_gen,
)


# ResNet 101

In [ ]:
def make_model_resnet101_mod(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    #base_model = tf.keras.applications.resnet.ResNet101(include_top=False, weights = "imagenet", input_shape = input_shape)
    base_model = tf.keras.applications.ResNet101V2(include_top=False, weights = None, input_shape = input_shape)
    x = inputs
    x = base_model(x)
    #print(x.shape)
    x = layers.BatchNormalization()(x)
    global_avg_layer = layers.GlobalAveragePooling2D()
    #x = global_avg_layer(x)
#     x = layers.Dropout(0.3)(x)
#     x = layers.Conv2D(1024, 1, activation='relu')(x)
#     x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(256, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Flatten()(x)
    #x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(num_classes, activation= "softmax")(x)
    base_model.trainable = True
    for layer in base_model.layers[:-5]:
        layer.trainable=False
    return tf.keras.Model(inputs,outputs)

In [ ]:
model = make_model_resnet101_mod(input_shape=(180,180,3) , num_classes= 5270)

In [ ]:
model.summary()

In [ ]:
import tensorflow_addons as tfa

In [ ]:
epochs = 100
base_learning_rate = 1e-3
INIT_LR = 1e-4
MAX_LR = 1e-3
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=base_learning_rate,
    decay_steps=50000,
    end_learning_rate = 1e-4,
    power=0.5)

steps_per_epoch = 7730
cyclic_lr_schedule = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate= MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size= 2
)

wandb.config = {
 "learning_rate" : lr_schedule,
 "epochs" : epochs,
 "batch_size" : 128
}

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=5, min_lr=1e-4)

optimizer = tf.keras.optimizers.Adam(learning_rate =  lr_schedule)
class LRLogger(tf.keras.callbacks.Callback):
    def __init__(self, optimizer):
        super(LRLogger, self).__init__
        self.optimizer =optimizer
    
    def on_epoch_end(self,epoch,logs):
        lr = self.optimizer.learning_rate(epoch)
        wandb.log({"lr":lr}, commit = False)
        
        
filepath = "./weights.best_{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_f = keras.callbacks.ModelCheckpoint(filepath,monitor = "val_accuracy", verbose=1, save_best_only=False, mode="max")

callbacks = [
    checkpoint_f,WandbCallback()
, LRLogger(optimizer) , reduce_lr]

model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy", 
    #using Focal cross entropy loss
    #loss = tfa.losses.SigmoidFocalCrossEntropy(),
    metrics=["accuracy", precision_m, recall_m, f1_m],
)

H = model.fit(
    train_gen, epochs=epochs, callbacks=callbacks, validation_data=val_gen,
)


## Inception V3

In [ ]:
def make_model_inceptionv3(input_shape, num_classes):
    inputs = keras.Input(shape = input_shape)
    base_model  = tf.keras.applications.InceptionV3(include_top=False, weights="imagenet", input_shape=input_shape)
    #base_model = tf.keras.applications.InceptionV3(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = True
    
    x = base_model(inputs)
    global_avg_layer = layers.GlobalAveragePooling2D()
    x = global_avg_layer(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation= "softmax")(x)
    base_model.trainable = True
    for layer in base_model.layers[:-5]:
        layer.trainable = False      # Train all layers - From Scratch
    return keras.Model(inputs,outputs)


In [ ]:
def make_model_inceptionv3_mod(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    base_model = tf.keras.applications.InceptionV3(include_top=False, weights = None, input_shape = input_shape)
    x = inputs
    x = base_model(x)
    #print(x.shape)
    x = layers.BatchNormalization()(x)
    global_avg_layer = layers.GlobalAveragePooling2D()
    #x = global_avg_layer(x)
    #x = layers.Flatten()(x)
    x = layers.Conv2D(512, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation= "softmax")(x)
    base_model.trainable = True
#     for layer in base_model.layers[:-20]:
#         layer.trainable=False
    return tf.keras.Model(inputs,outputs)

In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights = "imagenet", input_shape = (180,180,3))
#base_model.summary(line_length =100)


In [ ]:
model1 = make_model_inceptionv3_mod(input_shape = (180,180,3), num_classes = 5270)
#model = 

In [ ]:
model1.summary()

In [ ]:
model = make_model_inceptionv3_mod(input_shape = (180,180,3), num_classes = 5270)
#model = 

In [ ]:
model.summary()

In [ ]:
# WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 1934.0 batches). You may need to use the repeat() function when building your dataset.

In [ ]:
keras.utils.plot_model(model, show_shapes=True)
# Inception - V3 --> 

In [ ]:
import tensorflow_addons as tfa

In [ ]:


epochs = 100
base_learning_rate = 1e-3
INIT_LR = 1e-4
MAX_LR = 1e-3
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=base_learning_rate,
    decay_steps=50000,
    end_learning_rate = 1e-4,
    power=0.5)

steps_per_epoch = 7730
cyclic_lr_schedule = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate= MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size= 2
)

# cyclic_exp_lr_schedule = tfa.optimizers.ExponentialCyclicalLearningRate(
#     initial_learning_rate: Union[FloatTensorLike, Callable],
#     maximal_learning_rate: Union[FloatTensorLike, Callable],
#     step_size: tfa.types.FloatTensorLike,
#     scale_mode: str = 'iterations',
#     gamma: tfa.types.FloatTensorLike = 1.0,
#     name: str = 'ExponentialCyclicalLearningRate'
# )


wandb.config = {
 "learning_rate" : cyclic_lr_schedule,
 "epochs" : epochs,
 "batch_size" : 128
}

# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
#                               patience=3, min_lr=1e-4)

optimizer = tf.keras.optimizers.Adam(learning_rate = cyclic_lr_schedule)
class LRLogger(tf.keras.callbacks.Callback):
    def __init__(self, optimizer):
        super(LRLogger, self).__init__
        self.optimizer =optimizer
    
    def on_epoch_end(self,epoch,logs):
        lr = self.optimizer.learning_rate(epoch)
        wandb.log({"lr":lr}, commit = False)
        
        
filepath = "./weights.best_{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_f = keras.callbacks.ModelCheckpoint(filepath,monitor = "val_accuracy", verbose=1, save_best_only=False, mode="max")

callbacks = [
    keras.callbacks.ModelCheckpoint("inception_v3_training/save_at_{epoch}.h5"), checkpoint_f,WandbCallback()
, LRLogger(optimizer) ] #,reduce_lr ]

model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy", 
    #using Focal cross entropy loss
    #loss = tfa.losses.SigmoidFocalCrossEntropy(),
    metrics=["accuracy", precision_m, recall_m, f1_m],
)

H = model.fit(
    train_gen, epochs=epochs, callbacks=callbacks, validation_data=val_gen,
)



## Efficient Net B7

In [ ]:
def make_model_effnet_b7_1(input_shape, num_classes):
    inputs = keras.Input(shape = input_shape)
    base_model  = tf.keras.applications.EfficientNetB7(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = True
    
    x = base_model(inputs)
    global_avg_layer = layers.GlobalAveragePooling2D()
    x = global_avg_layer(x)
    x = layers.Dropout(0.2)(x)
#     x = layers.Conv2D(256, 1, activation='relu')(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.Dropout(0.3)(x)
#     x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation= "softmax")(x)
    base_model.trainable = False
    for layer in base_model.layers[:-5]:
        layer.trainable = False
    return keras.Model(inputs,outputs)


In [ ]:
def make_model_effnet_b7(input_shape, num_classes):
    inputs = keras.Input(shape = input_shape)
    base_model  = tf.keras.applications.EfficientNetB7(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = True
    
    x = base_model(inputs)
    global_avg_layer = layers.GlobalAveragePooling2D()
#     x = global_avg_layer(x)
#     x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(256, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(num_classes, activation= "softmax")(x)
    base_model.trainable = False
    for layer in base_model.layers[:-5]:
        layer.trainable = False
    return keras.Model(inputs,outputs)


In [ ]:
model = make_model_effnet_b7(input_shape = (180,180,3), num_classes = 5270)

In [ ]:
model.summary()

In [ ]:
## Step Decay 


In [ ]:
## Cyclical learning rate Approach
## --> Implement this later


In [ ]:
epochs = 50
base_learning_rate = 1e-3
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=base_learning_rate,
    decay_steps=80000,
    end_learning_rate = 1e-4,
    power=0.5)



wandb.config = {
 "learning_rate" : lr_schedule,
 "epochs" : epochs,
 "batch_size" : 128
}

earlystopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=5, min_lr=1e-4)

optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
class LRLogger(tf.keras.callbacks.Callback):
    def __init__(self, optimizer):
        super(LRLogger, self).__init__
        self.optimizer =optimizer
    
    def on_epoch_end(self,epoch,logs):
        lr = self.optimizer.learning_rate(epoch)
        wandb.log({"lr":lr}, commit = False)


In [ ]:
filepath = "./efficientnet_b7_training/weights.best_effnet_b7{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_f = keras.callbacks.ModelCheckpoint(filepath,monitor = "val_accuracy", verbose=1, save_best_only=False, mode="max")

callbacks = [checkpoint_f,WandbCallback()
, LRLogger(optimizer)]

model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy", precision_m, recall_m, f1_m],
)
H = model.fit(
    train_gen, epochs=epochs, callbacks=callbacks, validation_data=val_gen,
)

## NAS Net  - Large

In [ ]:
def make_model_nasnet_large(input_shape, num_classes):
    inputs = keras.Input(shape = input_shape)
    base_model  = tensorflow.keras.applications.nasnet.NASNetLarge(weights="imagenet", include_top=False)
    base_model.trainable = True
    
    x = base_model(inputs)
    global_avg_layer = layers.GlobalAveragePooling2D()
    x = global_avg_layer(x)
    #x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation= "softmax")(x)
    base_model.trainable = True
    for layer in base_model.layers[:-10]:
        layer.trainable = False
    return keras.Model(inputs,outputs)


In [ ]:
num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 128

# Tip: use ImageDataGenerator for data augmentation and preprocessing.
train_datagen = ImageDataGenerator()
train_gen_mod = BSONIterator(train_bson_file, train_images_df, train_offsets_df, 
                         num_classes, train_datagen, lock,target_size=(331,331),
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen_mod = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock,target_size=(331,331),
                       batch_size=batch_size, shuffle=True)

In [ ]:
model = make_model_nasnet_large(input_shape = (331,331,3), num_classes = 5270)

In [ ]:
model.summary()

In [ ]:
epochs = 10
base_learning_rate = 1e-2
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=base_learning_rate,
    decay_steps=20000,
    decay_rate=0.9)

wandb.config = {
 "learning_rate" : base_learning_rate,
 "epochs" : epochs,
 "batch_size" : 128
}

filepath = "./weights.best_nasnet_b7{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_f = keras.callbacks.ModelCheckpoint(filepath,monitor = "val_accuracy", verbose=1, save_best_only=False, mode="max")

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"), checkpoint_f, WandbCallback()
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate = base_learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy", precision_m, recall_m, f1_m],
)
H = model.fit(
    train_gen_mod, epochs=epochs, callbacks=callbacks, validation_data=val_gen_mod,
)


## Efficient Net V2-S - Under Exploration - NAS Type Model
### Keras nightly build

In [ ]:
!pip3 install effnetv2_model

In [ ]:
import tensorflow_hub as hub

In [ ]:
def make_model_efficientv2_s(input_shape, num_classes):
    inputs = keras.Input(shape = input_shape)
    hub_url= "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2"
    base_model = hub.KerasLayer(hub_url, trainable=False)
#     base_model.trainable = True
    
    x = base_model(inputs)
    global_avg_layer = layers.GlobalAveragePooling2D()
    x = global_avg_layer(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation= "softmax")(x)
#     for layer in base_model.layers[:-15]:
#         layer.trainable = False
    return keras.Model(inputs,outputs)


In [ ]:
print("Tensorflow Version:{}".format(tf.__version__))

In [ ]:
model = make_model_efficientv2_s(input_shape = (180,180,3), num_classes = 5270)

In [ ]:
model.summary()

In [ ]:
epochs = 10
base_learning_rate = 4e-3
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=base_learning_rate,
    decay_steps=20000,
    decay_rate=0.9)

wandb.config = {
 "learning_rate" : base_learning_rate,
 "epochs" : epochs,
 "batch_size" : 128
}


filepath = "./weights.best_effnetv2s_{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint_f = keras.callbacks.ModelCheckpoint(filepath,monitor = "val_accuracy", verbose=1, save_best_only=False, mode="max")

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"), checkpoint_f, WandbCallback()
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate = base_learning_rate),
    loss="categorical_crossentropy",
    metrics=["accuracy", precision_m, recall_m, f1_m],
)
model.fit(
    train_gen, epochs=epochs, callbacks=callbacks, validation_data=val_gen,
)
